In [8]:
%pwd

'd:\\Prg_Language\\Medical-Chatbot-Generative-_Ai'

In [9]:
import os
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Change directory to parent
os.chdir("../")

# Print current working directory
print(os.getcwd())

def load_pdf_file(data):
    loader = DirectoryLoader(data,
                             glob="*.pdf",
                             loader_cls=PyPDFLoader)
    documents = loader.load()
    return documents

# Use a try-except block to handle potential errors
try:
    extra_data = load_pdf_file(data='Data/')
    print(f"Successfully loaded {len(extra_data)} documents.")
except Exception as e:
    print(f"An error occurred: {str(e)}")

d:\Prg_Language
An error occurred: Directory not found: 'Data/'


In [10]:
def text_split(extra_data):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks = text_splitter.split_documents(extra_data)
    return text_chunks

In [11]:
text_chunk=text_split(extra_data)
print(f"Successfully loaded {len(text_chunk)} documents")

Successfully loaded 5860 documents


In [12]:
# text_chunk

In [1]:
from langchain.embeddings import HuggingFaceBgeEmbeddings
def download_hugging_face_embedding():
    embeddings= HuggingFaceBgeEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

c:\Users\asus\anaconda3\envs\medic-chatbot-env\lib\site-packages\requests\__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(


In [2]:
embeddings=download_hugging_face_embedding()

C:\Users\asus\AppData\Local\Temp\ipykernel_24204\2121797779.py:3: LangChainDeprecationWarning: The class `HuggingFaceBgeEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings= HuggingFaceBgeEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


In [17]:
import sentence_transformers
print(sentence_transformers.__version__)

4.1.0


In [18]:
query_result =embeddings.embed_query("hello world")
print("length: ", len(query_result))

length:  384


In [19]:
# query_result

In [20]:
from dotenv import load_dotenv
load_dotenv()

False

In [21]:
PINECONE_API_KEY=os.environ.get("PINECONE_API_KEY")
# OPENAI_API_KEY=os.environ.get("OPENAI_API_KEY")

In [25]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
import os

pc = Pinecone(api_key=PINECONE_API_KEY)
print(pc.list_indexes())
# Create a new index
index_name ="bot"
# pc.create_index(
#     name=index_name,
#     dimension=384,
#     metric="cosine",
#     spec=ServerlessSpec(
#         cloud="aws",
#         region="us-east-1",
#     )

# )

[{
    "name": "bot",
    "metric": "cosine",
    "host": "bot-y5z9xmf.svc.aped-4627-b74a.pinecone.io",
    "spec": {
        "serverless": {
            "cloud": "aws",
            "region": "us-east-1"
        }
    },
    "status": {
        "ready": true,
        "state": "Ready"
    },
    "vector_type": "dense",
    "dimension": 384,
    "deletion_protection": "disabled",
    "tags": null
}]


In [26]:
import os
os.environ["PINECONE_API_KEY"]=PINECONE_API_KEY

In [27]:
#embedding each chunk and upsert the embeddedings into ur pinecone index
from langchain_pinecone import PineconeVectorStore

docsearch= PineconeVectorStore.from_documents(
    documents=text_chunk,
    index_name=index_name,
    embedding=embeddings,
)

In [28]:
#exisiting index
from langchain_pinecone import PineconeVectorStore
docsearch= PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings,
)

In [29]:
docsearch

In [30]:
retriver=docsearch.as_retriever(search_type="similarity",search_kwargs={"k":3})

In [31]:
retriver_docs=retriver.invoke("what is acne")

In [32]:
retriver_docs


[Document(id='bce7b583-82e6-4510-97e0-76f25b6c3f7d', metadata={'creationdate': '2004-12-18T17:00:02-05:00', 'creator': 'PyPDF', 'moddate': '2004-12-18T16:15:31-06:00', 'page': 39.0, 'page_label': '40', 'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'source': 'Data\\Medicine_vol1.pdf', 'total_pages': 637.0}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 226\nAcne\nGEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 26'),
 Document(id='53003205-ec54-4b99-a7af-40d15d878999', metadata={'creationdate': '2004-12-18T17:00:02-05:00', 'creator': 'PyPDF', 'moddate': '2004-12-18T16:15:31-06:00', 'page': 39.0, 'page_label': '40', 'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'source': 'Data\\Medicine_vol1.pdf', 'total_pages': 637.0}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 226\nAcne\nGEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 26'),
 Document(id='8e4aa72c-2a75-4290-858e-6973642c4c5a', metadata={'creationdate': '2004-12-18T17:00:02-05:00', 'creator': 'PyPDF', 'moddate': '2004-12-18T16:15:31-06:00', 'page': 39.0

In [39]:
# from langchain_openai import OpenAI
# llm=OpenAI(temperature=0.4,max_tokens=500)
from langchain_community.llms import Ollama

llm = Ollama(model="gemma3:1b")
# response = llm.invoke("Hello, chatbot!")
# print(response)


In [40]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt=(
    "You are an assistant for question-answering tasks."
    "Use the following piece of retrived contex to answer"
    "the question. If you don't know the answer , say that you "
    "don't know. use three sentences maximum and keep the"
    "answer  concise."
    "\n\n"
    "{context}"
)

prompt =ChatPromptTemplate.from_messages(
    [
        ("system",system_prompt),
        ("human","{input}")
    ]
)

In [41]:
question_answer_chain = create_stuff_documents_chain(llm,prompt)
rag_chain = create_retrieval_chain(retriver,question_answer_chain)

In [43]:
response = rag_chain.invoke({"input": "what is Acromegaly and gigantism?"})
print(response["answer"])

Acromegaly is a condition where the pituitary gland releases an abnormal chemical, leading to excessive growth in bone and soft tissues, and various other physiological disturbances. This chemical is released from the brain and impacts multiple body systems. Acromegaly and gigantism are both related disorders caused by this chemical imbalance.
